**1. Install Required Libraries**

In [1]:
!pip install crewai
!pip install 'crewai[tools]'
!pip install langchain_groq

  Using cached langchain_core-0.1.52-py3-none-any.whl (302 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.11
    Uninstalling langchain-core-0.2.11:
      Successfully uninstalled langchain-core-0.2.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-groq 0.1.6 requires langchain-core<0.3,>=0.2.2, but you have langchain-core 0.1.52 which is incompatible.
  Using cached langchain_core-0.2.11-py3-none-any.whl (337 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.52
    Uninstalling langchain-core-0.1.52:
      Successfully uninstalled langchain-core-0.1.52
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.20 requires langchai

**2. Import Libraries and Set Environment Variables**

In [2]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from langchain_groq import ChatGroq
from google.colab import userdata

serper_api_key = userdata.get('SERPER_APIKEY')
groq_api_key = userdata.get('GROQ_APIKEY')

# Set environment variables (optional, for compatibility with some libraries)
os.environ["SERPER_API_KEY"] = serper_api_key
os.environ['GROQ_API_KEY'] = groq_api_key

In [3]:
# Initialize Groq API
llm = ChatGroq(model="gemma-7b-it", groq_api_key=groq_api_key)
print(llm.invoke("hi"))

# Initialize search tool (optional)
search_tool = SerperDevTool()

content="Hi! 👋 It's nice to hear from you. What would you like to talk about today? 😊" response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 15, 'total_tokens': 38, 'completion_time': 0.024669438, 'prompt_time': 0.00904107, 'queue_time': None, 'total_time': 0.033710508}, 'model_name': 'gemma-7b-it', 'system_fingerprint': 'fp_7d8efeb0b1', 'finish_reason': 'stop', 'logprobs': None} id='run-1ac19e2e-00a7-43fc-a5b1-21614e89c259-0' usage_metadata={'input_tokens': 15, 'output_tokens': 23, 'total_tokens': 38}


**3. Define Agents for FAQ Answering Workflow**

In [4]:
# Researcher agent gathers information about the smart TV
researcher = Agent(
    role="Smart TV Researcher",
    goal="Gather information about smart TVs based on a product description.",
    backstory=(
        "I'm an expert at finding relevant information about smart TVs. I can "
        "search the web and use my knowledge to understand product descriptions."
    ),
    llm=llm,
    max_iter=5,
    tools=[search_tool],
    allow_delegation=True,
    verbose=True,
)

# Writer agent formulates answers to FAQ questions
writer = Agent(
    role="Smart TV FAQ Writer",
    goal="Formulate clear and informative answers to FAQ questions about smart TVs "
         "using the provided information.",
    backstory=(
        "I can transform gathered information into well-structured and easy-to-understand "
        "answers to your questions about smart TVs."
    ),
    llm=llm,
    max_iter=5,
    allow_delegation=False,
)

**4. Define Tasks for the Workflow**

In [5]:
# Research task gathers information based on the product description
research_task = Task(
    description="Gather information about smart TVs from a product description.",
    expected_output="A list of FAQs and answers and Extracted information about the smart TV features",
    tools=[search_tool],
    agent=researcher,
)

# Write task formulates answers to FAQ questions
write_task = Task(
    description="Formulate answers to FAQ questions about smart TVs "
                 "using the gathered information.",
    expected_output="A list of answers to frequently asked questions",
    agent=writer,
    async_execution=False,
    output_file="smart_tv_faq.txt",
)

**5. Set up the crew and process for executing the tasks:**

In [6]:
# Define the crew and process
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_task],
    process=Process.sequential,
    max_rpm=3,
    cache=True,
)

**6. Execute the workflow with the necessary inputs:**

In [7]:
# Kickoff the workflow with product description and example FAQ questions
product_description = "This is a high-resolution smart TV with a sleek design, offering vibrant picture quality, built-in streaming apps, and voice assistant compatibility."
faq_questions = [
    "What are the different types of smart TV resolutions available?",
    "Does this smart TV have built-in streaming apps?",
    "How can I control this smart TV using voice commands?"
]

result = crew.kickoff(
    inputs={
        "product_description": product_description,
        "faq_questions": faq_questions,
    }
)
print(result)



> Entering new CrewAgentExecutor chain...
## Action: Search the internet
Action Input: {'search_query': 'Smart TV product description analysis'} 


Search results: Title: Smart TV Market Size, Share & Trends Analysis Report, 2030
Link: https://www.grandviewresearch.com/industry-analysis/smart-tv-industry
Snippet: The global smart TV market size was valued at USD 197.82 billion in 2022 and is expected to expand at a compound annual growth rate (CAGR) of 11.4% from ...
---
Title: What is smart TV? | Definition from TechTarget
Link: https://www.techtarget.com/whatis/definition/smart-TV
Snippet: A smart TV is a television that includes an internal processor and onboard storage and enables internet connectivity, similar to a smartphone or personal ...
---
Title: Smart TV Platforms Market Analysis Research Report [2023-2030]
Link: https://www.linkedin.com/pulse/smart-tv-platforms-market-analysis-research-report-mkckf
Snippet: The report focuses on the Smart TV Platforms market size, segmen